# Part1: Preprossing The Dataset


## Importing Libraries And Loading The Dataset

In [3]:
! pip install opendatasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
# Data Manipulation libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random


import opendatasets as od
import os

from bs4 import BeautifulSoup

import string
import re

from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer , TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier



In [7]:
import nltk
from nltk import PorterStemmer
from nltk import WordNetLemmatizer
from nltk import pos_tag
from nltk import word_tokenize
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [8]:
od.download_kaggle_dataset("https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews" , "data")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: mahmoudatiaeadatia
Your Kaggle Key: ··········


100%|██████████| 25.7M/25.7M [00:00<00:00, 75.3MB/s]


In [10]:
for dirpath, dirnames ,  filenames in os.walk("/content/data"):
    print(f"there is {len(dirnames)} dirs and {len(filenames)} files in {dirpath}")

there is 1 dirs and 0 files in /content/data
there is 0 dirs and 1 files in /content/data/imdb-dataset-of-50k-movie-reviews


In [11]:
df = pd.read_csv("/content/data/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv")
df_copy = df.copy()

## Exploring The Dataset

In [12]:
df_copy.head(10)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
5,"Probably my all-time favorite movie, a story o...",positive
6,I sure would like to see a resurrection of a u...,positive
7,"This show was an amazing, fresh & innovative i...",negative
8,Encouraged by the positive comments about this...,negative
9,If you like original gut wrenching laughter yo...,positive


In [13]:
df_copy.describe()

,review,sentiment
count,50000,50000
unique,49582,2
top,Loved today's show!!! It was a variety and not...,positive
freq,5,25000


I think we have some duplicated reviews in the dataset

In [14]:
df_copy["dub"] = df_copy.duplicated()

In [15]:
df_unique = df_copy[df_copy["dub"] == False]

In [16]:
df_unique.describe()

,review,sentiment,dub
count,49582,49582,49582
unique,49582,2,1
top,One of the other reviewers has mentioned that ...,positive,False
freq,1,24884,49582


In [17]:
df_unique["sentiment"].value_counts(), df_copy["sentiment"].value_counts()

(positive    24884
 negative    24698
 Name: sentiment, dtype: int64, positive    25000
 negative    25000
 Name: sentiment, dtype: int64)

In [18]:
df_copy.groupby("sentiment").count()

,review,dub
sentiment,,
negative,25000,25000
positive,25000,25000


I will work with df_copy because it is balaneced unlike df_unique

### Visualise Some Reviews

In [19]:
for _ in range(5):
  rand = random.randint(0,len(df_copy))
  senti = df_copy["sentiment"][rand]
  rev = df_copy["review"][rand]
  print(f"""
  The Review :
  {rev}
  Seniment:
  {senti}
  """)



  The Review :
  I realize why people hate this film. And, I hated Blair Witch Project,so go figure? This is about as staged as it gets & yes they do insult your intelligence by trying to make it seem real.I really liked the madame lalaurie storyline though it's more than likely made up. But, the main reason I like this film, is fake or not when the ghosts start attacking & kidnapping them,I get chill-bumps every time & have to look behind me as it feels like something's there with me.i know it's my imagination, but hey more than half of the drivel that is horror in today's cinemas & DVD's doesn't give me goosebumps,so that makes this a creepy delight.Not for everyone, as skeptics will hate it & not for gorehounds as with a PG-13 rating there is no gore. And, the females are very annoying!You'll wish the ghosts would take them off & experiment on them before it's all said & done. ** out of *****.
  Seniment:
  negative
  

  The Review :
  I had read an article about Dan Jansen's Olym

## Preprocessing The Texts

picking one value to test on

### Removing HTML Tags

In [20]:
value = df_copy.iloc[0]

In [21]:
value["review"]

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fa

In [ ]:
bs = BeautifulSoup(value["review"],"html.parser" )
value_no_html = bs.get_text()

### Removing Punctotion And Numbers

In [ ]:
value_no_html

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.I would say the main appeal of the show is due to the fact that it goes where other shows wo

In [ ]:
value_no_html.translate(str.maketrans("","", string.punctuation+string.digits))

'One of the other reviewers has mentioned that after watching just  Oz episode youll be hooked They are right as this is exactly what happened with meThe first thing that struck me about Oz was its brutality and unflinching scenes of violence which set in right from the word GO Trust me this is not a show for the faint hearted or timid This show pulls no punches with regards to drugs sex or violence Its is hardcore in the classic use of the wordIt is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary It focuses mainly on Emerald City an experimental section of the prison where all the cells have glass fronts and face inwards so privacy is not high on the agenda Em City is home to manyAryans Muslims gangstas Latinos Christians Italians Irish and moreso scuffles death stares dodgy dealings and shady agreements are never far awayI would say the main appeal of the show is due to the fact that it goes where other shows wouldnt dare Forget pretty picture

### Cleaning Function

In [ ]:
def clean_text(text):
  bs = BeautifulSoup(text,"html.parser" )
  value_no_html = bs.get_text()
  return value_no_html.translate(str.maketrans("","", string.punctuation+string.digits))

### Appling The clean_text Function

In [ ]:
df_copy["review"] = df_copy["review"].apply(lambda x : clean_text(x))

In [ ]:
for _ in range(5):
  rand = random.randint(0,len(df_copy))
  senti = df_copy["sentiment"][rand]
  rev = df_copy["review"][rand]
  print(f"""
  The Review :
  {rev}
  Seniment:
  {senti}
  """)
  


  The Review :
  Ok with this film there are a number of ingredients at workFirst put in loads of hillbilly truckersgood ole boys who have secret desire to take law into own hands Second put in evil hillbilly cops controlled by a Texas bred outlaw Third put in karate fighting trucker played by Chuck Norris Fourth put in a chump teenager as Norriss nephew captured by evil hillbilly cops Fifth show Chuck Norris fighting in slow motionNow mix all together and what have you gotYou guessed itone very mediocre movie
  Seniment:
  negative
  

  The Review :
  I consider myself a bit of a connoisseur of boxing movies and as such there is only one thing that prevents me from calling Gentleman Jim the best boxing movie ever made That is the Robert WisePaul Newman flick Somebody Up There Likes Me That movie might be number  but Gentleman Jim is a close number The movie doesnt just chronicle the rise of James J Corbett it also shows the sport of boxing at a crucial time of transition In the late

### Spilt to train test

In [ ]:
x_train , x_test , y_train , y_test = train_test_split(df_copy["review"],df_copy["sentiment"], test_size=0.2 , shuffle= True , )

# Part2: Modeling

#### Tokenization using CountVectorizer

In [ ]:
cv = CountVectorizer(
    lowercase= True,
    stop_words= "english",   
)
cv_train = cv.fit_transform(x_train)
cv_test = cv.transform(x_test)

In [ ]:
cv.get_stop_words()

frozenset({'a',
           'about',
           'above',
           'across',
           'after',
           'afterwards',
           'again',
           'against',
           'all',
           'almost',
           'alone',
           'along',
           'already',
           'also',
           'although',
           'always',
           'am',
           'among',
           'amongst',
           'amoungst',
           'amount',
           'an',
           'and',
           'another',
           'any',
           'anyhow',
           'anyone',
           'anything',
           'anyway',
           'anywhere',
           'are',
           'around',
           'as',
           'at',
           'back',
           'be',
           'became',
           'because',
           'become',
           'becomes',
           'becoming',
           'been',
           'before',
           'beforehand',
           'behind',
           'being',
           'below',
           'beside',
           'besides'

now we can create our first model

In [ ]:
log_reg = LogisticRegression()
log_reg.fit(cv_train , y_train)


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [ ]:
log_reg.score(cv_train , y_train)

0.99795

Evaluating the model

In [ ]:
log_reg.score(cv_test,y_test)

0.8859

In [ ]:
# same result
(log_reg.predict(cv_test)==y_test).sum()/len(y_test)

0.8859

In [ ]:
baseline = pd.DataFrame({"train":[log_reg.score(cv_train,y_train)], "test":[log_reg.score(cv_test,y_test)]} , index= ["baseline : just CountVectorizer"])

In [ ]:
baseline

,train,test
baseline : just CountVectorizer,0.99795,0.8859


## Model1 CountVectorizer using stemmer

---



In [ ]:
#creating a stemmer class to use it in sklearn CountVectorizer
ps = PorterStemmer()
class stemmer:
  def __call__(self, doc):
    tokens = word_tokenize(doc)
    return [ps.stem(word) for word in tokens]

In [ ]:
cv1 = CountVectorizer(
    lowercase= True,
    stop_words= "english",
    tokenizer= stemmer()
)
cv1_train = cv1.fit_transform(x_train)
cv1_test = cv1.transform(x_test)

/usr/local/lib/python3.8/dist-packages/sklearn/feature_extraction/text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'afterward', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'cri', 'describ', 'dure', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'formerli', 'forti', 'ha', 'henc', 'hereaft', 'herebi', 'hi', 'howev', 'hundr', 'inde', 'latterli', 'mani', 'meanwhil', 'moreov', 'mostli', 'nobodi', 'noon', 'noth', 'nowher', 'onc', 'onli', 'otherwis', 'ourselv', 'perhap', 'pleas', 'seriou', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'themselv', 'thenc', 'thereaft', 'therebi', 'therefor', 'thi', 'thu', 'togeth', 'twelv', 'twenti', 'veri', 'wa', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 'wherev', 'whi', 'yourselv'] not in stop_words.
  warnings.

KeyboardInterrupt: ignored

In [ ]:
'''counts = pd.DataFrame(cv1_test.toarray()[1],
                      columns=cv1.get_feature_names())'''

'counts = pd.DataFrame(cv1_test.toarray()[1],\n                      columns=cv1.get_feature_names())'

In [ ]:
# running the model
model_1 = LogisticRegression(max_iter = 1000)
model_1.fit(cv1_train , y_train)

LogisticRegression(max_iter=1000)

In [ ]:
model_1 = pd.DataFrame({"train":[log_reg.score(cv1_train,y_train)], "test":[log_reg.score(cv1_test,y_test)]} , index= ["Model 1 : CountVectorizer using Stemmer"])

In [ ]:
models = pd.concat([baseline, model_1])

In [ ]:
models

,train,test
baseline : just CountVectorizer,0.99795,0.8859
Model 1 : CountVectorizer using Stemmer,0.99795,0.8859


## Model2 CountVectorizer using lemmetizer


In [ ]:
wnl = WordNetLemmatizer()

In [ ]:
def lemma_pos(pos_tag):
  if pos_tag.startswith("N"):
    return "n"
  elif pos_tag.startswith("V"):
    return "v"
  elif pos_tag.startswith("J"):
    return "r"
  elif pos_tag.startswith("R"):
    return "r"
  else: return "n"

In [ ]:
class lemmatize:
  def __call__(self,doc):
    tokens = word_tokenize(doc)
    word_dict = pos_tag(tokens)
    return [wnl.lemmatize(word,lemma_pos(tag)) for word,tag in word_dict]

In [ ]:
cv2 = CountVectorizer(
    max_features= 10000,
    lowercase= True,
    tokenizer= lemmatize()
)
cv2_train = cv2.fit_transform(x_train)
cv2_test = cv2.transform(x_test)

KeyboardInterrupt: ignored

## Model3 TFIDF

In [ ]:
tf_idf = TfidfVectorizer(stop_words ="english")
tf_train = tf_idf.fit_transform(x_train)
tf_test = tf_idf.transform(x_test)

In [ ]:
model_3 = LogisticRegression(max_iter = 1000)
model_3.fit(tf_train , y_train)

LogisticRegression(max_iter=1000)

In [ ]:
model_3 = pd.DataFrame({"train":[model_3.score(tf_train,y_train)], "test":[model_3.score(tf_test,y_test)]} , index= ["Model 3 : TFIDFVectorizer "])

In [ ]:
models = pd.concat([models,model_3])

In [ ]:
models

,train,test
baseline : just CountVectorizer,0.99795,0.8859
Model 1 : CountVectorizer using Stemmer,0.99795,0.8859
Model 3 : TFIDFVectorizer,0.93355,0.8949


## Model 4 : 

In [ ]:
rf = RandomForestClassifier()
rf.fit(cv_train, y_train)

RandomForestClassifier()

In [ ]:
rf.score(cv_train, y_train)

1.0

In [ ]:
rf.score(cv_test, y_test)

0.7327

In [ ]:
rf = DecisionTreeClassifier()
rf.fit(cv_train, y_train)

DecisionTreeClassifier()

In [ ]:
rf.